In [4]:
import pandas as pd
import os
import librosa

storage_path = '/work/bbc6523/diverse_voices/'

In [4]:
dfs = {}

for file in os.listdir(storage_path + 'british_dialects_audio'):
    dfs[file] = pd.read_csv(storage_path + file + '.csv')

In [5]:
def add_audio_duration(row):
    audio, sr = librosa.load(storage_path + 'british_dialects_audio/' + file + '/'+ row['audio_file'] + ".wav", sr=None)  # sr=None to keep original sample rate

    # Duration in seconds
    duration = len(audio) / sr
    return duration

for file in dfs: 
    dfs[file]['audio_duration'] = dfs[file].apply(lambda row: add_audio_duration(row), axis=1)

In [7]:
subset_dfs = {}
for file in dfs:
    print(file)
    print(len(dfs[file]))
    dfs[file]['text_length'] = dfs[file]['text'].str.split().str.len()
    print(dfs[file]['text_length'].max())
    print(dfs[file]['text_length'].min())
    print(len(dfs[file][dfs[file]['text_length'] > 10]))
    print(dfs[file]['audio_duration'].max())
    print(dfs[file]['audio_duration'].min())
    print(len(dfs[file][dfs[file]['audio_duration'] > 7]))
    print('BRING TOGETHER')
    print(len((dfs[file][(dfs[file]['audio_duration'] >= 7) & (dfs[file]['text_length'] >= 10)])))
    #subset_dfs[file] = dfs[file][(dfs[file]['audio_duration'] >= 5) & (dfs[file]['text_length'] >= 5)]
    subset_dfs[file] = dfs[file][(dfs[file]['audio_duration'] >=5)]
    print('------------------------------------------')
    print()


british_dialects_southern_female
4161
27
2
2988
16.469333333333335
1.6213333333333333
1312
BRING TOGETHER
1306
------------------------------------------

british_dialects_northern_male
2097
27
2
1474
14.933333333333334
1.6213333333333333
651
BRING TOGETHER
641
------------------------------------------

british_dialects_southern_welsh_male
134
26
10
132
23.162666666666667
11.130666666666666
134
BRING TOGETHER
134
------------------------------------------

british_dialects_northern_female
750
27
2
521
14.677333333333333
1.7066666666666668
282
BRING TOGETHER
281
------------------------------------------

british_dialects_southern_welsh_female
98
26
8
92
23.674666666666667
12.666666666666666
98
BRING TOGETHER
94
------------------------------------------

british_dialects_southern_midlands_male
134
26
10
132
22.48
10.704
134
BRING TOGETHER
134
------------------------------------------

british_dialects_southern_northern_male
134
26
10
132
23.248
10.96
134
BRING TOGETHER
134
----------

In [37]:
file_list_male = [
'british_dialects_northern_male',
'british_dialects_southern_male',
'british_dialects_welsh_male',
'british_dialects_scottish_male',
'british_dialects_midlands_male']

In [43]:
file_list_female = ['british_dialects_southern_female',
'british_dialects_northern_female',
'british_dialects_midlands_female',
'british_dialects_welsh_female',
'british_dialects_scottish_female',]

In [8]:
file_list_male_female = ['british_dialects_southern_female', 'british_dialects_southern_male']

In [9]:
value_sets = [set(subset_dfs[file]['text']) for file in file_list_male_female]
common_values_female = set.intersection(*value_sets)
len(common_values_female)

1083

In [ ]:
value_sets = [set(subset_dfs[file]['text']) for file in file_list_male]
common_values_male = set.intersection(*value_sets)
len(common_values_male)

67

In [96]:
value_sets = [set(subset_dfs[file]['text']) for file in file_list_female]
common_values_female = set.intersection(*value_sets)
len(common_values_female)

49

In [7]:
import random
for file in subset_dfs: 
    subset_dfs[file]['__index'] = subset_dfs[file].index

grouped = {
    name: df.groupby(['text_length', 'audio_duration_rounded'])['__index'].apply(list)
    for name, df in subset_dfs.items()
}

# Step 3: Find shared keys (only those that exist in all DataFrames)
shared_keys = set.intersection(*(set(g.index) for g in grouped.values()))

# Step 4: Build matchable aligned rows across all DataFrames
matches = []
for key in shared_keys:
    min_count = min(len(grouped[name][key]) for name in grouped)
    for i in range(min_count):
        match = [grouped[name][key][i] for name in subset_dfs]
        matches.append(match)

if len(matches) < 100:
    raise ValueError(f"Only {len(matches)} matched rows available, can't sample 100.")

sampled_matches = random.sample(matches, 100)

# Step 6: Build filtered versions of the DataFrames
filtered_subset_dfs = {}
for df_idx, name in enumerate(subset_dfs):
    matched_indices = [match[df_idx] for match in sampled_matches]
    filtered_subset_dfs[name] = subset_dfs[name].loc[matched_indices].reset_index(drop=True)


/tmp/ipykernel_2581163/1161927865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_dfs[file]['__index'] = subset_dfs[file].index
/tmp/ipykernel_2581163/1161927865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_dfs[file]['__index'] = subset_dfs[file].index
/tmp/ipykernel_2581163/1161927865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [8]:
for file in filtered_subset_dfs:
    print(file)
    print(len(filtered_subset_dfs[file]))

british_dialects_southern_female
100
british_dialects_northern_male
100
british_dialects_northern_female
100
british_dialects_midlands_female
100
british_dialects_welsh_female
100
british_dialects_scottish_female
100
british_dialects_southern_male
100
british_dialects_welsh_male
100
british_dialects_scottish_male
100
british_dialects_midlands_male
100


In [ ]:
import numpy as np
import soundfile as sf
import time

def create_merged_file(row, standard_file_name, dialect_file_name):
    standard_audio, sr1 = librosa.load(storage_path + 'british_dialects_audio/' + standard_file_name + '/'+ row['audio_file_standard'] + ".wav", sr=None)
    dialect_audio, sr2 = librosa.load(storage_path + 'british_dialects_audio/' + dialect_file_name + '/'+ row['audio_file_dialect'] + ".wav", sr=None)

    # Create 2 seconds of silence
    pause_duration_sec = 2
    pause = np.zeros(int(sr1 * pause_duration_sec))
    if row['dialect_order'] == 'standard-dialect':
        joined = np.concatenate([standard_audio, pause, dialect_audio])
        file_name = str(row['index']) +'_'+ 'british_dialects_' + str(standard_file_name.replace('british_dialects_', '').replace('_male', '')) + '_' + str(dialect_file_name.replace('british_dialects_', '').replace('_male', ''))+ '_male'
    if row['dialect_order'] == 'dialect-standard':
        joined = np.concatenate([dialect_audio, pause, standard_audio])
        file_name = str(row['index'])+'_' + 'british_dialects_' + str(dialect_file_name.replace('british_dialects_', '').replace('_male', '')) + '_' + str(standard_file_name.replace('british_dialects_', '').replace('_male', ''))+ '_male'

    storage = storage_path +'british_dialects_audio/compare_audio/' + file_name + '.wav'
    os.makedirs(storage_path +'british_dialects_audio/compare_audio' , exist_ok=True)

    try:
        sf.write(storage, joined, sr1)
    except: 
        print(file_name)

    return file_name


In [114]:
base_file_male = 'british_dialects_southern_male'

for dialect_file_male in ['british_dialects_welsh_male']:
                         # 'british_dialects_midlands_male'
                        #'british_dialects_northern_male',
                        #'british_dialects_scottish_male',
                        #'british_dialects_welsh_male']:

   df1 = dfs[base_file_male].copy()
   df1 = df1[df1['text'].isin(list(common_values_male))]
   df1 = df1[df1['audio_duration'] < 11]
   df1 = df1.drop_duplicates(subset='text')
   print(len(df1))
   df2 = dfs[dialect_file_male].copy()
   df2 = df2[df2['text'].isin(list(common_values_male))]
   df2 = df2[df2['audio_duration'] < 11]
   df2 = df2.drop_duplicates(subset='text')
   print(len(df2))


   df1 = df1[['line_id', 'audio_file', 'text', 'text_length', 'audio_duration']]
   df2 = df2[['line_id', 'audio_file', 'text', 'text_length', 'audio_duration']]

   # Merge using this row ID
   merged = pd.merge(df1, df2, on='text', suffixes=('_standard', '_dialect'))

   merged['standard_file'] = base_file_male
   merged['dialect_file'] = dialect_file_male
   merged = merged.reset_index()

   merged['dialect_order'] = 'standard-dialect'
   merged_2 = merged.copy()

   merged_2['dialect_order'] = 'dialect-standard'

   full_df = pd.concat([merged, merged_2])

   full_df['audio_file'] = full_df.apply(lambda row: create_merged_file(row, base_file_male, dialect_file_male), axis=1)


   df_name = 'british_dialects_' + base_file_male.replace('british_dialects_', '').replace('_male', '') + '_'+ dialect_file_male.replace('british_dialects_', '').replace('_male', '') + '_male.csv'
   full_df.to_csv('merged_audio/' + df_name, index=False)

67
67


In [54]:
df1[df1['line_id'] == 'EN0021']

,line_id,audio_file,text,text_length,audio_duration
1,EN0021,som_00295_00472546698,This is a very common type of bow one showing ...,21,6.314667
192,EN0021,som_04310_01192796020,This is a very common type of bow one showing ...,21,8.192000
892,EN0021,som_05223_01459906245,This is a very common type of bow one showing ...,21,7.082667
970,EN0021,som_02121_00801090069,This is a very common type of bow one showing ...,21,7.338667
1270,EN0021,som_08886_00800439310,This is a very common type of bow one showing ...,21,6.570667
1330,EN0021,som_03034_00601255200,This is a very common type of bow one showing ...,21,8.362667
1611,EN0021,som_01523_00877829907,This is a very common type of bow one showing ...,21,6.058667
1627,EN0021,som_06136_00960050331,This is a very common type of bow one showing ...,21,7.850667
2081,EN0021,som_09799_01193920807,This is a very common type of bow one showing ...,21,6.741333
2392,EN0021,som_09697_02085046691,This is a very common type of bow one showing ...,21,6.912000


In [118]:
import numpy as np
import soundfile as sf
import time

def create_merged_file(row, standard_file_name, dialect_file_name):
    standard_audio, sr1 = librosa.load(storage_path + 'british_dialects_audio/' + standard_file_name + '/'+ row['audio_file_standard'] + ".wav", sr=None)
    dialect_audio, sr2 = librosa.load(storage_path + 'british_dialects_audio/' + dialect_file_name + '/'+ row['audio_file_dialect'] + ".wav", sr=None)

    # Create 2 seconds of silence
    pause_duration_sec = 2
    pause = np.zeros(int(sr1 * pause_duration_sec))
    if row['dialect_order'] == 'standard-dialect':
        joined = np.concatenate([standard_audio, pause, dialect_audio])
        file_name = str(row['index']) +'_'+ 'british_dialects_' + str(standard_file_name.replace('british_dialects_', '').replace('_female', '')) + '_' + str(dialect_file_name.replace('british_dialects_', '').replace('_female', ''))+ '_female'
    if row['dialect_order'] == 'dialect-standard':
        joined = np.concatenate([dialect_audio, pause, standard_audio])
        file_name = str(row['index'])+'_' + 'british_dialects_' + str(dialect_file_name.replace('british_dialects_', '').replace('_female', '')) + '_' + str(standard_file_name.replace('british_dialects_', '').replace('_female', ''))+ '_female'

    storage = storage_path +'british_dialects_audio/compare_audio/' + file_name + '.wav'
    os.makedirs(storage_path +'british_dialects_audio/compare_audio' , exist_ok=True)

    try:
        sf.write(storage, joined, sr1)
    except: 
        print(file_name)

    return file_name


base_file_female = 'british_dialects_southern_female'

for dialect_file_female in ['british_dialects_scottish_female']:
                         # 'british_dialects_midlands_female'
                        #'british_dialects_northern_female',
                        #'british_dialects_scottish_female',
                        #'british_dialects_welsh_female']:

   df1 = dfs[base_file_female].copy()
   df1 = df1[df1['text'].isin(list(common_values_female))]
   df1 = df1[df1['audio_duration'] < 11]
   df1 = df1.drop_duplicates(subset='text')
   print(len(df1))
   df2 = dfs[dialect_file_female].copy()
   df2 = df2[df2['text'].isin(list(common_values_female))]
   df2 = df2[df2['audio_duration'] < 11]
   df2 = df2.drop_duplicates(subset='text')
   print(len(df2))


   df1 = df1[['line_id', 'audio_file', 'text', 'text_length', 'audio_duration']]
   df2 = df2[['line_id', 'audio_file', 'text', 'text_length', 'audio_duration']]

   # Merge using this row ID
   merged = pd.merge(df1, df2, on='text', suffixes=('_standard', '_dialect'))

   merged['standard_file'] = base_file_female
   merged['dialect_file'] = dialect_file_female
   merged = merged.reset_index()

   merged['dialect_order'] = 'standard-dialect'
   merged_2 = merged.copy()

   merged_2['dialect_order'] = 'dialect-standard'

   full_df = pd.concat([merged, merged_2])

   full_df['audio_file'] = full_df.apply(lambda row: create_merged_file(row, base_file_female, dialect_file_female), axis=1)


   df_name = 'british_dialects_' + base_file_female.replace('british_dialects_', '').replace('_female', '') + '_'+ dialect_file_female.replace('british_dialects_', '').replace('_female', '') + '_female.csv'
   full_df.to_csv('merged_audio/' + df_name, index=False)

49
49


In [10]:
subset

NameError: name 'subset' is not defined

In [30]:
import numpy as np
import soundfile as sf
import time

def create_merged_file_fm(row, base_file_female, base_file_male):
    female_audio, sr1 = librosa.load(storage_path + 'british_dialects_audio/' + base_file_female + '/'+ row['audio_file_female'] + ".wav", sr=None)
    male_audio, sr2 = librosa.load(storage_path + 'british_dialects_audio/' + base_file_male + '/'+ row['audio_file_male'] + ".wav", sr=None)

    # Create 2 seconds of silence
    pause_duration_sec = 2
    pause = np.zeros(int(sr1 * pause_duration_sec))
    if row['gender_order'] == 'female-male':
        joined = np.concatenate([female_audio, pause, male_audio])
        file_name = str(row['index']) +'_'+ 'british_dialects_southern_fm'
    if row['gender_order'] == 'male-female':
        joined = np.concatenate([male_audio, pause, female_audio])
        file_name = str(row['index'])+'_' + 'british_dialects_southern_mf'

    storage = storage_path +'british_dialects_audio/compare_audio/' + file_name + '.wav'
    os.makedirs(storage_path +'british_dialects_audio/compare_audio' , exist_ok=True)

    try:
        sf.write(storage, joined, sr1)
    except: 
        print(file_name)

    return file_name


base_file_female = 'british_dialects_southern_female'

base_file_male = 'british_dialects_southern_male'


value_sets = [set(subset_dfs[file]['text']) for file in file_list_male_female]
common_values_female = set.intersection(*value_sets)
len(common_values_female)

df1 = dfs[base_file_female].copy()
df1 = df1[df1['text'].isin(list(common_values_female))]
df1 = df1[df1['audio_duration'] >= 5]
df1 = df1[df1['text_length'] >= 5]
df1 = df1.drop_duplicates(subset='text')
df1 = df1.drop_duplicates(subset='line_id')
df1 = df1.sample(n=101, random_state=1)
print(len(df1))
df2 = dfs[base_file_male].copy()
df2 = df2[df2['text'].isin(list(common_values_female))]
df2 = df2[df2['audio_duration'] >= 5]
df2 = df2[df2['text_length'] >= 5]
df2 = df2.drop_duplicates(subset='text')
df2 = df2.drop_duplicates(subset='line_id')
print(len(df2))

df1 = df1[['line_id', 'audio_file', 'text', 'text_length', 'audio_duration']]
df2 = df2[['line_id', 'audio_file', 'text', 'text_length', 'audio_duration']]

# Merge using this row ID
merged = pd.merge(df1, df2, on='text', suffixes=('_female', '_male'))
print(len(merged))

merged['female_file'] = base_file_female
merged['male_file'] = base_file_male

merged = merged.reset_index()

merged['gender_order'] = 'female-male'

merged_2 = merged.copy()

merged_2['gender_order'] = 'male-female'

full_df = pd.concat([merged, merged_2])

full_df['audio_file'] = full_df.apply(lambda row: create_merged_file_fm(row, base_file_female, base_file_male), axis=1)


df_name = 'british_dialects_southern_gender.csv'
full_df.to_csv('merged_audio/' + df_name, index=False)





101
1071
100


In [13]:
len(df2.line_id.unique())

1071

## Filter dataset

In [9]:
storage_path = '/work/bbc6523/diverse_voices/'

files = ['british_dialects_scottish_male.csv',
 'british_dialects_midlands_male.csv',
 'british_dialects_midlands_female.csv',
 'british_dialects_welsh_male.csv',
 'british_dialects_southern_female.csv',
 'british_dialects_southern_male.csv',
 'british_dialects_scottish_female.csv',
 'british_dialects_northern_male.csv',
 'british_dialects_northern_female.csv',
 'british_dialects_welsh_female.csv']

In [10]:
dfs = {}
for file in files:
    dfs[file] = pd.read_csv(storage_path + 'data/' + file)

In [14]:
def add_audio_duration(row):
    audio, sr = librosa.load(storage_path + 'british_dialects_audio/' + file[:-4] + '/'+ row['audio_file'] + ".wav", sr=None)  # sr=None to keep original sample rate

    # Duration in seconds
    duration = len(audio) / sr
    return duration

for file in dfs: 
    dfs[file]['audio_duration'] = dfs[file].apply(lambda row: add_audio_duration(row), axis=1)

In [20]:
subset_dfs = {}
for file in dfs:
    print(file)
    subset_dfs[file] = dfs[file][(dfs[file]['audio_duration'] >=4)]

british_dialects_scottish_male.csv
british_dialects_midlands_male.csv
british_dialects_midlands_female.csv
british_dialects_welsh_male.csv
british_dialects_southern_female.csv
british_dialects_southern_male.csv
british_dialects_scottish_female.csv
british_dialects_northern_male.csv
british_dialects_northern_female.csv
british_dialects_welsh_female.csv


In [26]:
value_sets = [set(subset_dfs[file]['text']) for file in files]
common_values_text = set.intersection(*value_sets)
len(common_values_text)

50

In [24]:
value_sets = [set(subset_dfs[file]['line_id']) for file in files]
common_values = set.intersection(*value_sets)
len(common_values)

51

In [53]:
file_df['audio_file']

3      /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_00036278823.wav
7      /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_00495324118.wav
10     /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_00407944176.wav
12     /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_01518530037.wav
14     /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_01289814182.wav
15     /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_00263357569.wav
16     /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_01407056263.wav
18     /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_welsh_female/wef_12484_01739883713.wav
24     /work/bbc6523/diverse_voices/british_dialects_audio/british_dialects_wels

In [52]:
all_dfs = []
for file in subset_dfs:
    file_df = subset_dfs[file][subset_dfs[file]['text'].isin(list(common_values_text))]
    file_df = file_df[['line_id', 'audio_file', 'text', 'audio_duration']]
    file_df['audio_file'] = '/work/bbc6523/diverse_voices/british_dialects_audio/' +file[:-4] + '/'+ file_df['audio_file'] + '.wav'
    print(len(file_df))
    file_df = file_df.drop_duplicates(subset='text')
    print(len(file_df))
    file_df['gender'] = file.split('_')[-1][:-4]
    file_df['accent'] = file.split('_')[-2]
    all_dfs.append(file_df)
full_df = pd.concat(all_dfs)

524
50
138
50
78
50
525
50
1298
50
1344
50
279
50
660
50
233
50
385
50


In [54]:
full_df.to_csv('british_dialects.csv', index=False)